In [1]:
from	google.colab	import	drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
data = pd.read_csv("/content/drive/MyDrive/Datasets/Dataset .csv")
data.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [21]:
data['Cuisines'] = data['Cuisines'].fillna('Unknown')

In [22]:
# Combine relevant features into a single string for content based filtering
data['Combined Features'] = data['Cuisines'] + " " + data['City'] + " " + data['Price range'].astype(str)

In [23]:
print(data[['Restaurant Name', 'Combined Features']].head())

          Restaurant Name                                  Combined Features
0        Le Petit Souffle           French, Japanese, Desserts Makati City 3
1        Izakaya Kikufuji                             Japanese Makati City 3
2  Heat - Edsa Shangri-La  Seafood, Asian, Filipino, Indian Mandaluyong C...
3                    Ooma                 Japanese, Sushi Mandaluyong City 4
4             Sambo Kojin                Japanese, Korean Mandaluyong City 4


In [24]:
# Convert text to a count matrix
count_vectorizer = CountVectorizer(stop_words='english')
count_matrix = count_vectorizer.fit_transform(data['Combined Features'])

In [25]:
# Calculate the cosine similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [26]:
# Display the shape of the similarity matrix
print("Cosine Similarity Matrix Shape:", cosine_sim.shape)

Cosine Similarity Matrix Shape: (9551, 9551)


In [27]:
# Function to recommend restaurants
def recommend_restaurants(restaurant_name, cosine_sim=cosine_sim):
    # Get the index of the restaurant that matches the name
    idx = data[data['Restaurant Name'] == restaurant_name].index[0]

    # Get similarity scores for all restaurants with this one
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort restaurants by similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top 10 most similar restaurants
    top_indices = [i[0] for i in sim_scores[1:11]]

    # Return the names of the top 10 restaurants
    return data['Restaurant Name'].iloc[top_indices]

In [28]:
# Test the function with a restaurant name
print("Recommended Restaurants for 'Le Petit Souffle':")
print(recommend_restaurants('Le Petit Souffle'))

Recommended Restaurants for 'Le Petit Souffle':
1                            Izakaya Kikufuji
3                                        Ooma
4                                 Sambo Kojin
480              Fuji Bay Japanese Restaurant
4993                          Tokyo Mon Amour
9329              Hippopotamus - Museum Hotel
9334                                 wagamama
18                 Hobing Korean Dessert Cafe
485                              Johnnie Mars
491     Tokyo Japanese Steakhouse & Sushi Bar
Name: Restaurant Name, dtype: object


In [29]:
# Example user input for preferences
user_cuisine = "Japanese"
user_city = "Makati City"
user_price_range = 3

# Filter dataset based on user preferences
filtered_data = data[
    (data['Cuisines'].str.contains(user_cuisine, case=False)) &
    (data['City'] == user_city) &
    (data['Price range'] == user_price_range)
]

# Recommend top 5 restaurants from the filtered data
if not filtered_data.empty:
    print("Top Recommendations Based on Your Preferences:")
    print(filtered_data[['Restaurant Name', 'Cuisines', 'City', 'Price range']].head())
else:
    print("No recommendations match your preferences.")


Top Recommendations Based on Your Preferences:
    Restaurant Name                    Cuisines         City  Price range
0  Le Petit Souffle  French, Japanese, Desserts  Makati City            3
1  Izakaya Kikufuji                    Japanese  Makati City            3
